# Neighborhood Computations

In [1]:
import numpy as np
# import igl
import meshplot

ModuleNotFoundError: No module named 'igl'

In [466]:
bunny_v, bunny_f = igl.read_triangle_mesh("data/bunny.off")
cube_v, cube_f = igl.read_triangle_mesh("data/cube.obj")
sphere_v, sphere_f = igl.read_triangle_mesh("data/sphere.obj")

In [467]:
meshplot.plot(bunny_v, bunny_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [468]:
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [469]:
meshplot.plot(sphere_v, sphere_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Vertex-to-Face Relations

In [470]:
def VtF(V, F):
    l, n = igl.vertex_triangle_adjacency(F, V.size)

    for i in range(n.size):
        if n[i] >= l.size: break
        print(f"Vertex {i}:")

        s = "Faces: "
        al = []

        for j in range(n[i+1]-n[i]):
            al.append(l[n[i]+j])
        print(s + f"{al}")
            
    return

## Vertex-to-Vertex Relations

In [471]:
def VtV(V):
    l = igl.adjacency_list(V)
    print(l)

    for i in range(len(l)):
        faces = []
        print(f"Vertex adjacent to vertex {i}:")
        for j in l[i]:
            faces.append(j)
        print(faces)
        
    return

In [590]:
# neighborhood relations

VtF(bunny_v, bunny_f)
print()
VtV(bunny_f)

Vertex 0:
Faces: [849, 850, 912, 944, 945]
Vertex 1:
Faces: [738, 788, 1278, 1833]
Vertex 2:
Faces: [248, 2795, 2857, 2972, 3408]
Vertex 3:
Faces: [911, 912, 945, 960, 974, 991]
Vertex 4:
Faces: [971, 2977, 2984, 3012, 3013]
Vertex 5:
Faces: [1455, 1658, 1706, 2471, 4816, 4884]
Vertex 6:
Faces: [1906, 2443, 2506, 2518, 2526]
Vertex 7:
Faces: [5, 15, 103, 2576, 2697, 2799]
Vertex 8:
Faces: [55, 118, 144, 2711, 3005]
Vertex 9:
Faces: [2599, 2729, 2805, 2945, 3066]
Vertex 10:
Faces: [96, 941, 2736, 2773, 2803, 2832, 3164, 3165]
Vertex 11:
Faces: [13, 96, 1082, 1689, 2736, 3137, 3143]
Vertex 12:
Faces: [1689, 3137, 3816, 3911]
Vertex 13:
Faces: [1037, 1053, 1054, 1087, 1122, 1133, 1134]
Vertex 14:
Faces: [90, 107, 123, 139]
Vertex 15:
Faces: [1017, 1036, 1053, 1074, 1121, 1122]
Vertex 16:
Faces: [712, 713, 728, 737, 750, 1970, 2034]
Vertex 17:
Faces: [680, 713, 728, 3078]
Vertex 18:
Faces: [407, 515, 2113, 2114, 2246, 2335, 3402]
Vertex 19:
Faces: [99, 100, 164, 209, 884, 886, 984]
Vertex 

## Shading

Meshplot requires per vertex normals, so we need to "explode" the mesh

In [473]:
def explode(V, F):
    e_v = np.copy([V[0]])
    e_f = np.copy(F)

    # checked = []
    vertex = 0
    
    index_i = 0
    for i in F:
        index_j = 0
        
        for j in i:            
            e_f[index_i][index_j] = vertex
            if vertex == 0: 
                index_j += 1
                vertex +=1
                continue
            
            index_j += 1
            vertex += 1
            e_v = np.append(e_v, [np.copy(V[j])], axis=0)
        index_i += 1
    
    return e_v, e_f

v, f = explode(cube_v, cube_f)

### Flat Shading

In [474]:
def flat(V, F):
    Z = np.zeros(3)
    return igl.per_face_normals(V, F, Z)

v = sphere_v
f = sphere_f
n = flat(v, f)
meshplot.plot(v, f, n=n, shading={"flat": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-vertex Shading

In [475]:
def vertex(V, F):
    return igl.per_vertex_normals(V, F)

v = sphere_v
f = sphere_f
n = vertex(v, f)
meshplot.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-corner Shading

In [476]:
def corner(V, F):
    e_v, e_f = explode(V, F)
    N = igl.per_corner_normals(V, F, 20.)
    return N, e_v, e_f

v = bunny_v
f = bunny_f
n, v, f = corner(v, f)
meshplot.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

## Connected Components

In [587]:
def con_com(F):
    cc = igl.face_components(F)
    print(np.unique(cc).size)
    meshplot.plot(v, f, c=cc)
    return

v, f = igl.read_triangle_mesh("data/car.off")
con_com(f)

v, f = igl.read_triangle_mesh("data/coffeecup.off")
con_com(f)

11


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.194198…

2


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-3.253649…

## A simple subdivision scheme

In [593]:
import math

def subdivision(V, F):
    # step 1
    vn = V.shape[0]
    M = igl.barycenter(V, F)
    new_v = np.append(V, M, axis=0)

    index = 0
    new_face = np.empty((1,3))
    for face in F:
        a, b, c = face
        new_face = np.append(new_face, [[a, b, vn + index]], axis = 0)
        new_face = np.append(new_face, [[c, b, vn + index]], axis = 0)
        new_face = np.append(new_face, [[a, c, vn + index]], axis = 0)
        index += 1
        
    # step 2
    p = V.copy()
    l = igl.adjacency_list(F)
    index = 0
    for x in l:
        n = len(x)
        a = (4 - 2 * math.cos(2 * math.pi / n)) / 9
        v = V[index]
        
        vi = np.sum(V[x], axis = 0)
        p[index] = (1-a)*v + (a/n)*vi
        
        nv = np.vstack((p, M))
        index += 1

    #step 3
    t, to = igl.triangle_triangle_adjacency(F)    
    index = 0
    nf = np.empty((1,3))
    for face in F:
        t0 = t[index][0]
        t1 = t[index][1]
        t2 = t[index][2]

        for tri in t[index]:
            for ver in F[tri]:
                if ver in face:
                    nf = np.append(nf, [[ver, vn+index, tri+vn]], axis=0)
        
        index+=1
    return nv, nf

v, f = igl.read_triangle_mesh("data/sphere.obj")
v, f = subdivision(v, f)
meshplot.plot(v, f, shading={"wireframe": True})

v, f = igl.read_triangle_mesh("data/bumpy.off")
v, f = subdivision(v, f)
meshplot.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.9802322…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

V = cube_v.copy()
F = cube_f.copy()

M = igl.barycenter()
v_pp = np.row_stack((V, M))

ev, fe, ef = igl.edge_topology(v, f)

offset = V.shape[0]
f_pp = []

for fi, es in enumerate(fe):
    m_fi_i = fi + offset

    for ei in es:
        e = ev[ei]
        f_pp.append([e[0], e[1], m_fi_i])

f_pp = np.array(f_pp)

v_p = v_pp.copy()
vv = igl.adjacency_list(F)

for i in range(V.shape[0]):
    neighs = vv[i]
    n = len(neighs)
    an = (4-2 * cos(2*pi/n)) / 9
    p = (1-an) * V[i]
    for j in neighs:
        p+=an/n*V[j]

ev, fe, ef = igl.edge_topology(F-pp)

for ei, e in enumerate(ev):
    if e[0] >= V.shape[0] or e[1] >= V.shape[0]:
        continue

    f0 = ef[ei, 0]
    f1 = ef[ei, 1]

    vf0 = f_pp[f0]
    vf1 = f_pp[f1]

    vf0s = set(vf0)
    vf1s = set(vf1)

    es = set(e)
    Ls = vf0s - es
    Rs = vf1s - es

    L = Ls.pop()
    R = Rs.pop()

    f_p[f0] = [e[0], R, L]
    f_p[f1] = [e[1], R, L]
